In [1]:
import torch
from config_loader import load_config, get_model_name
from datasets import load_dataset
from plotting import plot_conditioned_examples

config = load_config("VAE")
# Load dataset
train_loader, val_loader, test_loader = load_dataset(config["data_params"])

model_name = get_model_name(config)

x, x_cond, y = next(iter(train_loader))

In [3]:
from lightning_extensions import ExtendedTrainer
from modules import VAEModule

model = VAEModule(config['model_params'], model_name=config['model_name'])

trainer = ExtendedTrainer(project_name="MultiTaskVariationalAutoecnoders_test", **config['trainer_params'], model_name=model_name)
trainer.fit(model, train_loader, val_loader)
#save
trainer.save_model_checkpoint()

NameError: name 'model_name' is not defined